#### Dati di implementazione
- Mappe di feature
- Dimensione della tabella di hash per ogni feature (forse si può dedurre)

#### Scissione
Un bucket che supera una certa taglia viene scisso in questo modo:
1. Viene trovata la feature tra le k disponibili che rappresenta meglio i valori nel bucket, cioè quella che assume i valori più disparati (per ora è scelta a caso)
2. Viene creata una tabella di hash di taglia pari alla dimensione fornita dall'utente (table_sizes)
3. Alla tabella è associata una funzione di hash appropriata
4. Tutti i valori del bucket sono spostati in questa tabella
5. Il bucket è eliminato e sostituito da un puntatore alla tabella

In [41]:
import random
import copy
import traceback

# Facciamo che ogni bucket ha la stessa dimensione massima
MAXBUCKETSIZE = 10

In [48]:
class NodeTable():
    # insertion_function combina estrazione della feature e hashing
    # Tree contiene un puntatore alla struttura HashTree per quando bisogna prelevare una feature (da rifare)
    def __init__(self, size, fm, fm_index, hash_f, hashtree):
        self.size = size
        self.table = [[] for x in range(size)]
        
        self.feature_map = fm
        self.feature_map_index = fm_index
        self.hash = hash_f
        
        self.hashtree = hashtree
    
    def insert(self, element):
        # print(element, self.size, self.insertion_function(element))
        
        # Scegli la destinazione in base all'hash
        index = self.hash(self.feature_map(element))
        cell = self.table[index]
        
        # Devo distinguere se in una data posizione c'è un bucket (list)
        # o un puntatore ad un altro nodo (NodeTable)
        if type(cell) is list: # Bucket
            cell.append(element)
            
            # Scindi il bucket se è troppo grande
            if len(cell) > MAXBUCKETSIZE:
                self.scission(index)
        elif type(cell) is NodeTable: # Pointer
            # Ripeti ricorsivamente nel nodo figlio
            cell.insert(element)
        else:
            print("UNKNOWN TYPE")
            exit()
        

    def scission(self, index):
        # print("Scissione all'indice {}".format(index))
        cell = copy.copy(self.table[index])

        # Per ora prende una feature a caso
        feature_map, hash_f, table_size, feature_map_index = self.hashtree.random_feature(cell, self.feature_map_index)

        if len(self.table) == 1:
            # Caso in cui è presente solo la radice (da fare meglio, sostanzialmente ripete __init__)
            self.size = table_size
            self.table = [[] for x in range(table_size)]
            
            self.feature_map = feature_map
            self.feature_map_index = feature_map_index
            self.hash = hash_f
            
            for value in cell:
                self.insert(value)
        else:
            newChild = NodeTable(table_size, feature_map, feature_map_index, hash_f, self.hashtree)
            for value in cell:
                newChild.insert(value)
                                
            self.table[index] = newChild
        # print("Done rearranging")
    
    def search(self, featured_set):
        short = featured_set.get(self.feature_map_index)
        if short is None:
            return sum([cell if type(cell) is list else cell.search(featured_set) for cell in self.table], [])
        dest = self.table[self.hash(short)]
        return dest if type(dest) is list else dest.search(featured_set)
    
    def __str__(self):
        return "NodeTable\nSize: {}\nTable: {}".format(self.size, self.table)
        
    def prettyPrint(self, depth=0):
        result = "{} NodeTable (feature {}) [{}] =>\n".format(self.feature_map_index, depth, len(self.table))
        for cell in self.table:
            result += "--- " * (depth+1)
            if type(cell) is list:
                result += "[{}] => {}".format(len(cell), cell)
            elif type(cell) is NodeTable:
                result += "() => {}".format(cell.prettyPrint(depth + 1))
            else:
                print("UNKNOWN TYPE")
                exit()
            result += "\n"
        return result

In [49]:
class HashTree():
    def __init__(self, feature_maps, table_sizes):
        # Gli hash sono da fare meglio
        # (mappa feature, hash per la tabella, taglia della tabella)
        self.features = list(zip(feature_maps, [lambda x, size=size: hash(x) % size for size in table_sizes], table_sizes))
        
        # Numero di feature disponibili
        self.feature_amount = len(feature_maps)

        # Il primo nodo non cerca una feature particolare e utilizza un solo bucket
        self.tree = NodeTable(1, lambda x: x, -1, lambda x: 0, self)
        
    def insert(self, element):
        self.tree.insert(element)
        
    def normalized_gini(self, ls):
        unique = set(ls)
        freqs = [ls.count(u) / len(unique) for u in unique]
        gini = 1 - sum([ f**2 for f in freqs ])
        return gini * len(unique) / (len(unique) - 1)
    
    def scattering_index(self, feature, ls):
        return self.normalized_gini([feature(value) for value in ls])
        
    def random_feature(self, bucket, old_index):
        evalutation = [(index, feature, self.scattering_index(feature[0], bucket)) for index, feature in enumerate(self.features) if index != old_index]
        index, new_feature = max(evalutation, key=lambda t: t[2])[:2]
        return *new_feature, index
    
    def visit(self):
        return self.tree.search(dict())
    
    # featured_set {dict index => value} = list of values for features
    def search(self, featured_set):
        return self.tree.search(featured_set)

    def __str__(self):
        return self.tree.prettyPrint()

### Test

In [53]:
FEATURE_AMOUNT=3
DATA_SIZE=3
DATA_AMOUNT=200

test_data = [[random.randint(1, 100) for y in range(DATA_SIZE)] for x in range(DATA_AMOUNT)]

#feature_maps = [
#    lambda x: x[1] * 2 + x[2],
#    lambda x: x[0] ** 2,
#    lambda x: x[2] + x[1] + x[0],
#    lambda x: x[1] ** x[0],
#    lambda x: x[2] + x[2] + x[1]
#]
feature_maps = [
    lambda x: x[0],
    lambda x: x[1],
    lambda x: x[2]
]
table_sizes = [random.randint(3, 10) for x in range(FEATURE_AMOUNT)]

print(table_sizes)

ht = HashTree(feature_maps, table_sizes)

count = 0
try:
    for test in enumerate(test_data):
        ht.insert(test)
        count += 1
except RecursionError:
    print(count)
    traceback.print_exc()    

# Test che nessun dato sia andato disperso    
sorted(ht.visit()) == sorted(test_data)

[3, 8, 6]


TypeError: unhashable type: 'list'

In [37]:
print(ht)

0 NodeTable (feature 0) [7] =>
--- [7] => [[98, 50, 15], [84, 5, 85], [35, 48, 49], [49, 30, 50], [28, 67, 45], [77, 54, 84], [7, 96, 1]]
--- () => 1 NodeTable (feature 1) [3] =>
--- --- [7] => [[99, 36, 65], [92, 72, 21], [57, 78, 66], [43, 78, 75], [29, 69, 85], [36, 87, 60], [92, 6, 45]]
--- --- [10] => [[8, 40, 8], [71, 46, 10], [15, 88, 76], [29, 1, 88], [1, 19, 91], [57, 49, 30], [1, 31, 6], [15, 100, 29], [57, 19, 85], [57, 82, 79]]
--- --- [3] => [[22, 32, 85], [85, 20, 91], [43, 92, 1]]

--- () => 1 NodeTable (feature 1) [3] =>
--- --- [4] => [[100, 18, 62], [86, 93, 16], [65, 66, 100], [2, 57, 54]]
--- --- [5] => [[100, 34, 19], [65, 40, 93], [51, 1, 39], [51, 28, 45], [93, 76, 91]]
--- --- [11] => [[93, 59, 71], [30, 8, 64], [30, 59, 32], [65, 65, 90], [100, 68, 90], [23, 5, 94], [30, 47, 80], [44, 65, 32], [72, 35, 81], [44, 59, 91], [93, 14, 65]]

--- () => 1 NodeTable (feature 1) [3] =>
--- --- [5] => [[66, 81, 44], [87, 45, 54], [45, 39, 15], [38, 36, 26], [17, 66, 100]]

In [107]:
index = 0
example, feature = test_data[index], feature_maps[index]
print(example)

res = ht.search(dict([(index, feature(example))]))
len(res), res

[63, 92, 50]


(23,
 [[71, 77, 88],
  [55, 56, 11],
  [31, 77, 67],
  [31, 70, 89],
  [79, 28, 2],
  [63, 92, 50],
  [47, 29, 46],
  [79, 50, 14],
  [71, 92, 21],
  [63, 50, 42],
  [71, 29, 10],
  [39, 43, 9],
  [87, 2, 40],
  [23, 44, 52],
  [7, 23, 74],
  [95, 23, 9],
  [79, 52, 24],
  [79, 33, 84],
  [39, 82, 49],
  [23, 61, 100],
  [7, 82, 29],
  [7, 12, 94],
  [95, 27, 32]])